In [7]:
import pandas as pd
from bokeh.io import output_notebook, push_notebook, notebook
import networkx as nx
from bokeh.plotting import figure, from_networkx, show
import ipywidgets as widgets
from ipywidgets import interact


output_notebook()

Loading BokehJS ...

In [8]:
df = pd.read_csv('D13_graph4.csv',sep=';')
df = df.astype('int32')
# df

In [9]:
weights = df[df['ID2'] == 17]['weight'].tolist()
for i in range(len(weights)):
    if i == 16:
        df.loc[(df['ID1'] == 17) & (df['ID2'] == i+2), 'weight'] = weights[i]
    else: 
        df.loc[(df['ID1'] == 17) & (df['ID2'] == i+1), 'weight'] = weights[i]

# df[df['ID1'] == 17]

In [10]:
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes,NodesAndLinkedEdges,
                          HoverTool, MultiLine, Plot, TapTool, SetValue)
from bokeh.palettes import Spectral4

G = nx.from_pandas_edgelist(df=df,source='ID1', target='ID2', edge_attr=['weight'])


p = Plot(width= 700, height = 700, title = 'Nodes are clickable and you can hover over edges and nodes')
p.grid.grid_line_color = None

graph = from_networkx(G, nx.spring_layout,scale=1.8, center=(0,0))

p.add_tools(TapTool(),BoxSelectTool())



p.add_tools(HoverTool(renderers=[graph.edge_renderer],tooltips=[('Node 1', '@start'), ('Node 2', '@end'), ('Weight', '@weight')]))
p.add_tools(HoverTool(renderers=[graph.node_renderer], tooltips=[('Node', '@index')]))

graph.node_renderer.glyph = Circle(size=20, fill_color=Spectral4[0])
graph.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])
graph.node_renderer.data_source.data = dict(
    index = list(range(1,len(G)+1))
)

graph.edge_renderer.glyph = MultiLine(line_color="#808080", line_alpha=0.5, line_width=2)
graph.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=2)
graph.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=2)

graph.selection_policy = NodesAndLinkedEdges()
graph.inspection_policy = EdgesAndLinkedNodes()


p.renderers.append(graph)

edge_data_source = graph.edge_renderer.data_source.data

def update_weight(threshold):
    ds = {
        'start': [start for start, end, weight in zip(edge_data_source['start'], edge_data_source['end'], edge_data_source['weight']) if weight > threshold],
        'end': [end for start, end, weight in zip(edge_data_source['start'], edge_data_source['end'], edge_data_source['weight']) if weight > threshold],
        'weight': [weight for weight in edge_data_source['weight'] if weight > threshold]
    }
    graph.edge_renderer.data_source.data = ds
    push_notebook(handle=handle)



handle = show(p, notebook_handle=True)

In [11]:
greater_than_slider = widgets.IntSlider(value=0, min=0, max = 5, description='Weight:', disabled=False)

interact(update_weight, threshold=greater_than_slider)

interactive(children=(IntSlider(value=0, description='Weight:', max=5), Output()), _dom_classes=('widget-inter…

<function __main__.update_weight(threshold)>